#### 1. Data Review & Cleaning

In [81]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
pd.set_option('display.max_columns', None)

In [82]:
# Read dataset
df = pd.read_csv("fifa21_male2.csv")

/home/sabina/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [117]:
# Have a quick look at the data:
df.head()

,name,age,overall_score,nationality,club,best_overall,best_position,position,potential_score,team_&_contract,height,weight,preferred_foot,growth,joined,value,wage,release_clause,contract,attacking,crossing,finishing,heading_accuracy,short_passing,volleys,skill,dribbling,curve,fk_accuracy,long_passing,ball_control,movement,acceleration,sprint_speed,agility,reactions,balance,power,shot_power,jumping,stamina,strength,long_shots,mentality,aggression,interceptions,positioning,vision,penalties,composure,defending,marking,standing_tackle,sliding_tackle,goalkeeping,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes,total_stats,base_stats,weak_foot,skill_moves,away_win,draw_games,international_reputation,pace,shooting,passing,dribble,defend,physical,hits,left_striker,striker,right_striker,left_winger,left_forward,center_forward,right_forward,right_winger,left_attacking_midfield,center_attacking_midfield,right_attacking_midfield,left_midfielder,left_center_midfield,center_midfield,right_center_midfield,right_midfielder,left_wing_back,left_defensive_midfield,center_defensive_midfield,right_defensive_midfield,right_wing_back,left_back,left_center_back,center_back,right_center_back,right_back,goalkeeper
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,G. Pasquale,33,69,Italy,Udinese,71,LWB,LM,69,Udinese 2008 ~ 2016,"6'0""",181lbs,Left,0,"Jul 1, 2008",€625K,€7K,€0,2008 ~ 2016,313,75,50,59,71,58.0,338,73,65.0,60,69,71,347,68,74,68.0,69,68.0,347,74,68.0,69,68,68,320,72,69.0,63.0,66.0,50,NaN,208,70,69,69.0,56,14,5,15,10,12,1929,408,3 ★,2★,Medium,High,2 ★,71,59,70,71,68,69,4,65+0,65+0,65+0,68+0,67+0,67+0,67+0,68+0,68+0,68+0,68+0,69+0,69+0,69+0,69+0,69+0,71+-2,70+-1,70+-1,70+-1,71+-2,70+-1,69+0,69+0,69+0,70+-1,17+0
16,Luis García,37,71,Spain,KAS Eupen,70,CM,CM CAM CDM,71,KAS Eupen 2014 ~ 2019,"5'10""",143lbs,Right,0,"Jul 19, 2014",€600K,€7K,€1.1M,2014 ~ 2019,337,68,64,61,76,68.0,369,69,79.0,79,71,71,305,56,50,62.0,65,72.0,324,75,54.0,64,60,71,362,71,71.0,72.0,73.0,75,79.0,153,70,43,40.0,56,9,12,13,11,11,1906,385,4 ★,3★,Medium,Medium,1 ★,53,69,73,69,58,63,4,67+1,67+1,67+1,67+0,68+0,68+0,68+0,67+0,70+1,70+1,70+1,68+1,70+1,70+1,70+1,68+1,62+1,66+1,66+1,66+1,62+1,60+1,60+1,60+1,60+1,60+1,17+1
27,J. Cole,33,71,England,Coventry City,71,CAM,CAM RM RW LM,71,Coventry City 2016 ~ 2020,"5'9""",161lbs,Right,0,"Jan 7, 2016",€1.1M,€15K,€0,2016 ~ 2020,337,80,64,41,77,75.0,387,79,84.0,77,69,78,295,48,42,71.0,59,75.0,284,72,58.0,29,56,69,317,69,39.0,69.0,74.0,66,NaN,99,35,34,30.0,51,9,6,13,16,7,1770,354,4 ★,4★,Medium,Low,2 ★,45,68,76,77,36,52,11,64+0,64+0,64+0,70+0,69+0,69+0,69+0,70+0,71+0,71+0,71+0,68+0,66+0,66+0,66+0,68+0,52+0,54+0,54+0,54+0,52+0,47+0,46+0,46+0,46+0,47+0,15+0
36,D. Yorke,36,68,Trinidad &amp; Tobago,Sunderland,70,ST,NaN,82,Sunderland 2009,"5'11""",165lbs,Right,14,NaN,€0,€0,€0,2009,264,54,70,60,80,NaN,255,68,NaN,46,64,77,176,59,62,NaN,55,NaN,239,63,NaN,51,66,59,271,59,70.0,72.0,NaN,70,NaN,75,34,41,NaN,68,5,21,64,21,21,1348,369,3 ★,1★,NaN,NaN,1 ★,61,66,66,69,47,60,3,67+0,67+0,67+0,66+0,67+0,67+0,67+0,66+0,70+0,70+0,70+0,66+0,68+0,68+0,68+0,66+0,56+0,65+0,65+0,65+0,56+0,57+0,51+0,51+0,51+0,57+0,22+0
41,Iniesta,36,81,Spain,Vissel Kobe,82,CAM,CM CAM,81,Vissel Kobe 2018 ~ 2021,"5'7""",150lbs,Right,0,"Jul 16, 2018",€5.5M,€12K,€7.2M,2018 ~ 2021,367,75,69,54,90,79.0,408,85,80.0,70,83,90,346,61,56,79.0,75,75.0,297,67,40.0,58,62,70,370,58,70.0,78.0,93.0,71,89.0,181,68,57,56.0,45,6,13,6,13,7,2014,420,4 ★,4★,High,Medium,4 ★,58,70,85,85,63,59,149,72+3,72+3,72+3,79+0,79+0,79+0,79+0,79+0,82+-1,82+-1,82+-1,79+2,81+0,81+0,81+0,79+2,70+3,73+3,73+3,73+3,70+3,67+3,64+3,64+3,64+3,67+3,17+3


We notice immediately that the columns `Player Photo`, `Club Logo`, and `Flag Photo` contain *sofifa* links, which will not help in the current analysis, so we can drop them.

In [85]:
df.drop(['Player Photo', 'Club Logo', 'Flag Photo'], axis=1, inplace=True)

We also notice there is an ID column which we can use as our index column, after ensuring it doesn't have any duplicate values:

In [88]:
# If the number of unique values in the ID column are the same as the number of rows
# in the dataframe, we can set it as the index.
print(len(df['ID'].unique())/df.shape[0])

1.0


In [89]:
df.set_index('ID', inplace=True, drop=True)

We also notice that there are many columns which contain abbreviations that are unfamiliar for a non-player, so we will rename the columns to include the full names of the features. We will do so by using a `.csv` file containing the values for the abbreviations as seen on the *sofifa* website and standardize the column names alongside:

In [99]:
# Change capitalization to lowercase and replace spaces with underscores:
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ", "_")

In [101]:
# Read positions.csv into dataframe
positions = pd.read_csv('positions.csv', header=None, index_col=0)

# Change dataframe to series and then dictionary so it can be used to rename columns:
positions = positions.squeeze().to_dict()

# Change the column names:
for column in df:
    if column in positions:
        df.rename(columns=positions, inplace=True)

We can quickly check how many columns have more than 75% of null values, so we can discard them from our analysis:

In [109]:
def check_null_values(df, threshold=75):
    nulls_percentage = {}
    for column in df:
        number_of_nulls = df[column].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[0], 1)
        if null_percentage >= threshold:
            nulls_percentage[column] = null_percentage
    return nulls_percentage

check_null_values(df)

{}

In [108]:
# As `loan_date_end` has mostly `NaN` values, we can discard it:
df.drop('loan_date_end', axis=1, inplace=True)

We can have a quick look at the maximum percentage of `NaN` values in any column for reference:

In [110]:
def max_nulls(df):
    nulls_percentage = []
    for column in df:
        number_of_nulls = df[column].isna().sum()
        null_percentage = round(number_of_nulls * 100 / df.shape[0], 1)
        nulls_percentage.append(null_percentage)
    return max(nulls_percentage)

max_nulls(df)

2.5

As the maximum amount of nulls in any column is 2.5%, we can continue with our initial review of the data. We can have a look at the number of unique values per column to see if there are any columns that have only one value:

In [112]:
def check_unique_values(df):
    single_value_columns = []
    for column in df:
        if len(df[column].unique()) == 1:
            single_value_columns.append(column)
    return single_value_columns

check_unique_values(df)

['gender']

In [114]:
# Check the unique values in the gender column
df['gender'].unique()

array(['Male'], dtype=object)

In [116]:
# Remove the Gender column as the data shows only male players
df.drop('gender', axis=1, inplace=True)

We can see the `team_&_contract` column seems to have the same information as the `club` & `contract` columns, so we might be able to remove it as well after checking that our assumption is correct.

In [120]:
# Create a joined column to compare to the team & contract column
df['club_&_contract'] = df['club'] + ' ' + df['contract']

In [124]:
print(df['club_&_contract'].head())
print(df['team_&_contract'].head())

ID
2           Udinese 2008 ~ 2016
16        KAS Eupen 2014 ~ 2019
27    Coventry City 2016 ~ 2020
36              Sunderland 2009
41      Vissel Kobe 2018 ~ 2021
Name: club_&_contract, dtype: object
ID
2           Udinese 2008 ~ 2016
16        KAS Eupen 2014 ~ 2019
27    Coventry City 2016 ~ 2020
36              Sunderland 2009
41      Vissel Kobe 2018 ~ 2021
Name: team_&_contract, dtype: object


In [148]:
# Check that the columns are identical
def check_identical_columns(col1, col2, df):
    diff_values = pd.DataFrame({col1:[], col2:[]})
    identical = 1

    for i in range(0, df.shape[0]):
        if df[col1].iloc[i] == df[col2].iloc[i]:
            continue
        else:
            diff_values.loc[len(diff_values.index)] = [df[col1].iloc[i], df[col2].iloc[i]]
            identical = 0
            

    if identical == 0:
        print(diff_values)
    else:
        return 'Columns are identical.'         
            
check_identical_columns(col1='club_&_contract', col2='team_&_contract', df=df)         

                         club_&_contract            team_&_contract
0                                    NaN               England Free
1                                    NaN                France Free
2                                    NaN             Australia Free
3                                    NaN        Czech Republic Free
4                                    NaN         United States Free
..                                   ...                        ...
208                                  NaN                 India Free
209  Côte d'Ivoire FC Zürich 2019 ~ 2023      FC Zürich 2019 ~ 2023
210   New Zealand Rio Ave FC 2020 ~ 2024     Rio Ave FC 2020 ~ 2024
211   Slovenia SK Rapid Wien 2020 ~ 2023  SK Rapid Wien 2020 ~ 2023
212                                  NaN  United Arab Emirates Free

[213 rows x 2 columns]


In [125]:
# Check null values in columns:
print(df['club_&_contract'].isna().sum())
print(df['team_&_contract'].isna().sum())

23
0


In [149]:
# Check if columns are identical after removing the rows with null values:
df_test = df.copy()
df_test.dropna(axis=0, how="any", subset=['club_&_contract'], inplace=True)


check_identical_columns(col1='club_&_contract', col2='team_&_contract', df=df_test)

                             club_&_contract            team_&_contract
0    Chievo Verona Bari Jun 30, 2018 On Loan  Bari Jun 30, 2018 On Loan
1     Fenerbahçe SK Norwich City 2012 ~ 2015   Norwich City 2012 ~ 2015
2              Portugal FC Porto 2019 ~ 2021       FC Porto 2019 ~ 2021
3            Mexico Club América 2019 ~ 2024   Club América 2019 ~ 2024
4              Paraguay U.N.A.M. 2003 ~ 2021       U.N.A.M. 2003 ~ 2021
..                                       ...                        ...
185           Slovenia FC Zürich 2019 ~ 2022      FC Zürich 2019 ~ 2022
186         Bolivia Club Bolívar 2020 ~ 2024   Club Bolívar 2020 ~ 2024
187      Côte d'Ivoire FC Zürich 2019 ~ 2023      FC Zürich 2019 ~ 2023
188       New Zealand Rio Ave FC 2020 ~ 2024     Rio Ave FC 2020 ~ 2024
189       Slovenia SK Rapid Wien 2020 ~ 2023  SK Rapid Wien 2020 ~ 2023

[190 rows x 2 columns]


In [69]:
# We can remove Team & Contract column, as the same information is present in the Club
# and Contract columns
df.drop('Team & Contract', axis=1, inplace=True)

In [72]:
star_columns = ['weak_foot', 'skill_moves', 'international_rating']

# Check unique values
for column in star_columns:
    print(df[column].unique())

# Select only the first character from the string
for column in star_columns:
    df[column] = df[column].str[0]

# Convert data type from object to integer
for column in star_columns:
    df[column] = pd.to_numeric(df[column], errors='raise')

# Check that the transformation was successful
for column in star_columns:
    print(df[column].dtypes)

KeyError: 'weak_foot'

In [73]:
# Explore the contract data
df['contract'].unique()

# As the end of the contract is typically represented by the last 4 characters of the 
# strings, we will extract those where possible:
def clean_contract(x):
    try:
        x = int(x[-4:])
    except:
        pass
    return x

df['contract'] = df['contract'].apply(clean_contract)


In [ ]:
# We'll now check what non-integer values remained in the column:
df['Contract'].unique()

In [ ]:
# As there are many values ending with 'On Loan', we'll first check that these are not
# equivalent to the joined date by looking at some data samples. Additionally,
# we also want to see if the "Country Free" is related to the nationality of the player:
print(df.loc[:, ['Contract', 'Joined', 'Nationality']].head(50))
print(df.loc[:, ['Contract', 'Joined', 'Nationality']].tail(50))

In [74]:
# We'll now remove the 'On Loan' string from the Contract column to extract the year and
# replace all 'Country Free' values with NaN:
def clean_loans(x):
    try:
        x = int(x.replace(" On Loan", "")[-4:])
    except:
        x = np.nan
    return x    

df['Contract'] = df['Contract'].apply(clean_loans)

In [80]:
# Check that the operation was successful
df['Contract'].unique()


array([  nan, 2018., 2014., 2021., 2015., 2019., 2017., 2020., 2016.,
       2022., 2023.])